In [1]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import gymnasium as gym
from agents import SACAgent2
from omegaconf import DictConfig, OmegaConf
from hydra import compose, initialize
import matplotlib.pyplot as plt
from IPython.display import clear_output
import os
import wandb
import torch as T
from gymnasium.wrappers import RecordVideo
from EnvironmentUtils import LunarEnvHypercubeFabric, LunarEnvRandomFabric, LunarEnvFixedFabric
from agents import SACAgent2, DDPGAgent, SACAgent
import numpy as np
from utils import *
os.environ["SDL_VIDEODRIVER"] = "dummy"

In [2]:
import wandb
api = wandb.Api()
run = api.run("tum-adlr-ws22-06/ADLR randomized envs/2eqd07v7")

In [3]:
load_best_model(get_agent_from_run_cfg(run.config), run)

c:\Users\volob\.conda\envs\adlr\lib\site-packages\gymnasium\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
wandb:   5 of 5 files downloaded.  


Best model loaded from .\artifacts\lunar_lander_model-v483 with reward 128.606


In [41]:
#hist = run.history()
hist = run.history(keys=["Validation after episode", "Average evaluation reward"])

Unable to load pandas, call history with pandas=False


In [54]:
model_artifacts = [ x for x in  run.logged_artifacts() if x.type == "model" ]

In [55]:
[x.name for x in model_artifacts]

['lunar_lander_model:v441',
 'lunar_lander_model:v443',
 'lunar_lander_model:v445',
 'lunar_lander_model:v446',
 'lunar_lander_model:v449',
 'lunar_lander_model:v452',
 'lunar_lander_model:v455',
 'lunar_lander_model:v459',
 'lunar_lander_model:v463',
 'lunar_lander_model:v467',
 'lunar_lander_model:v471',
 'lunar_lander_model:v474',
 'lunar_lander_model:v479',
 'lunar_lander_model:v483',
 'lunar_lander_model:v486',
 'lunar_lander_model:v490',
 'lunar_lander_model:v493',
 'lunar_lander_model:v497']

In [53]:
run.logged_artifacts()[0].type

'run_table'

In [42]:
hist

[{'_step': 3,
  'Validation after episode': 0,
  'Average evaluation reward': -181.12},
 {'_step': 8558,
  'Validation after episode': 50,
  'Average evaluation reward': -117.105},
 {'_step': 14998,
  'Validation after episode': 100,
  'Average evaluation reward': -139.532},
 {'_step': 21479,
  'Validation after episode': 150,
  'Average evaluation reward': -173.549},
 {'_step': 45611,
  'Validation after episode': 200,
  'Average evaluation reward': -103.319},
 {'_step': 88429,
  'Validation after episode': 250,
  'Average evaluation reward': -16.015},
 {'_step': 131514,
  'Validation after episode': 300,
  'Average evaluation reward': -32.373},
 {'_step': 175925,
  'Validation after episode': 350,
  'Average evaluation reward': -14.947},
 {'_step': 220530,
  'Validation after episode': 400,
  'Average evaluation reward': 31.971},
 {'_step': 266049,
  'Validation after episode': 450,
  'Average evaluation reward': -9.123},
 {'_step': 313795,
  'Validation after episode': 500,
  'Avera

In [34]:
for i in hist:
    if i['Average evaluation reward'] != None:
        

TypeError: 'SampledHistoryScan' object is not subscriptable

In [32]:
[x for x in hist if 'Average evaluation reward' in x]

KeyboardInterrupt: 

In [24]:
hist[-17]

{'_step': 565160,
 'Batch': 774,
 '_runtime': 8675.511723995209,
 'Training episode': 774,
 'train_critic_loss': 8.716723442077637,
 'train_actor_loss': -57.18047332763672,
 '_timestamp': 1671029519.16284}

In [9]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [2]:
api = wandb.Api()
artifact = api.artifact('tum-adlr-ws22-06/ADLR randomized envs/lunar_lander_model:v131')

In [ ]:
def get_model(wandb_artifact, model_class, input_env = False):
    model = wandb_artifact.get(model_name)
    return model

In [6]:
artifact.download()

wandb:   5 of 5 files downloaded.  


'.\\artifacts\\lunar_lander_model-v131'

In [7]:
with initialize(version_base=None, config_path="conf"):
    cfg = compose(config_name="config")

In [10]:
env_fabric = LunarEnvHypercubeFabric(env_params=cfg.env, pass_env_params=True)
env = env_fabric.generate_env()
env_info = {"input_dims":env.observation_space.shape, "n_actions": env.action_space.shape[0], "max_action": env.action_space.high}

c:\Users\volob\.conda\envs\adlr\lib\site-packages\gymnasium\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [11]:
agent = SACAgent2(**OmegaConf.to_object(cfg.agent), **OmegaConf.to_object(cfg.training),
                      **env_info)
agent.load_agent(".\\artifacts\\lunar_lander_model-v131")

RuntimeError: Error(s) in loading state_dict for SACActorNetwork:
	size mismatch for fc1.weight: copying a param with shape torch.Size([256, 8]) from checkpoint, the shape in current model is torch.Size([256, 12]).

In [19]:
@widgets.interact_manual(g=(-12.0, 0.0), wind_power=(0.0, 20.0), turbulence_power=(0.0, 2.0)
                         )
def make_video(g=-10, wind_power=0.0, turbulence_power=0.0, plot_every_action=5):
    env = gym.make('LunarLander-v2', continuous=True, render_mode='rgb_array', gravity=g , enable_wind=True, wind_power=wind_power, 
             turbulence_power=turbulence_power)
    obs, info = env.reset()
    step = 0
    while True:
        action = agent.action(obs, addNoise=False)
        new_obs, reward, done, _, _ = env.step(action)
        if step % plot_every_action == 0:
            clear_output(wait=True)
            plt.imshow( env.render())
            plt.show()
        step += 1
        if done:
            break
    env.close()

interactive(children=(FloatSlider(value=-10.0, description='g', max=0.0, min=-12.0), FloatSlider(value=0.0, de…

In [ ]:
def validate(agent, validation_args, experiment_path, episode, test_env_fabric):
    '''
    doing all the validation stuff + logging
    returns, whether the env is solved
    '''
    
    for evaluation_episode in range(validation_args.eval_eps):
        video_path = os.path.join(experiment_path, "videos", str(episode))
        test_env = test_env_fabric.generate_env()
        gravity, enable_wind, wind_power, turbulence_power = test_env.gravity, test_env.enable_wind, test_env.wind_power, test_env.turbulence_power
        test_env = RecordVideo(test_env, video_path)
        obs, info = test_env.reset()
        rewards = 0

        for step in range(validation_args.validation_episode_length):

            # Get deterministic action
            with T.no_grad():
                action = agent.action(obs, addNoise=False)
                

            # Take step in environment
            new_obs, reward, done, _, _ = test_env.step(action)

            # Update obs
            obs = new_obs

            # Update rewards
            rewards += reward
            stop_reward.append(rewards)

            # End episode if done
            if done:
                break

    
    avg_reward = round(sum(stop_reward) / len(stop_reward), 3)
    min_reward = round(min(stop_reward), 3)
    
    if validation_args.eval_stop_condition == "avg":  
        stop_reward = avg_reward
    elif validation_args.eval_stop_condition == "min":
        stop_reward = min_reward
    else:
        raise ValueError(f"Unknown eval_stop_condition {validation_args.eval_stop_condition}")
    
    save_path = os.path.join(experiment_path, "saves")
    
    agent.save_agent(save_path)
    
    
    art = wandb.Artifact("lunar_lander_model", type="model")
    for f in os.listdir(save_path):
        art.add_file(os.path.join(save_path, f))
    wandb.log_artifact(art)
    
    
    print(f"Episode: {episode} | Average evaluation reward: {avg_reward} | Min evaluation reward: {min_reward} | Agent saved at {save_path}")
    
    wandb.log({"Validation after episode": episode,  "Average evaluation reward": avg_reward,
               "Min evaluation reward": min_reward})
    with open(f"{experiment_path}/evaluation_rewards.csv", "a") as f:
        f.write(f"{episode}, {stop_reward}\n")
    try:
        if stop_reward > test_env.spec.reward_threshold * 1.1:  # x 1.1 because of small eval_episodes
            print(f"Environment solved after {episode} episodes")
            return True
    except Exception as e:
        if stop_reward > -120:
            print(f"Environment solved after {episode} episodes")
            return True
    return False